In [1]:
import tensorflow as tf

In [104]:
def compute():
    a = tf.Variable(5.0,name='a')
    print(a)

In [107]:
# same as above (no matter True or False)
def compute():
    with tf.variable_scope("t", reuse=True) as scope:
        a = tf.Variable(5.0,name='a')
        print(a)

In [110]:
# get_variable is totally different
def compute():
    a = tf.get_variable(name='a', initializer=5.0, dtype=tf.float32)
    print(a)

In [113]:
# same as above
def compute():
    with tf.variable_scope("t") as scope: # reuse=False
        a = tf.get_variable(name='a', initializer=5.0, dtype=tf.float32)
        print(a)

In [119]:
# test simple
tf.reset_default_graph()
compute()
tf.get_variable_scope().reuse_variables()  
compute()

<tf.Variable 't/a:0' shape=() dtype=float32_ref>
<tf.Variable 't/a:0' shape=() dtype=float32_ref>


In [26]:
# 1. get(X) get(Y) okay, but cannot share parameters in high-level API, i.e.,
# if you use tensorboard, you can see that they did not share kernel or bias
# 2. get(X) reuse get(Y), not okay
# ValueError: Variable dense_1/kernel does not exist, or was not created with tf.get_variable()
def get(X):
    layer = tf.layers.dense(inputs=X,units=1024)
    print layer

In [29]:
# this is different (high-level API), see below:
# "ValueError: Variable discriminator/dense/kernel already exists, disallowed"
# 1. get(X) get(Y) not okay, since
#    (1) if reuse = True, the first is wrong
#    (2) if reuse = False, the second is wrong
# 2. get(X) reuse get(X), can share the parameters in high-level API, i.e.,
# if you use tensorboard, you can see that they share the same kernel and bias
def get(X):
    with tf.variable_scope("discriminator"): # "reuse = False" is default
        layer = tf.layers.dense(inputs=X,units=1024)
        print layer

In [31]:
# test simple
tf.reset_default_graph()
X = tf.placeholder(shape=(None,1024), dtype=tf.float32)
Y = tf.placeholder(shape=(None,1024), dtype=tf.float32)
Z = tf.placeholder(shape=(None,1024), dtype=tf.float32)
get(X)
tf.get_variable_scope().reuse_variables()  
get(Y)
get(Z)
# get(X)
# get(Y)

Tensor("discriminator/dense/BiasAdd:0", shape=(?, 1024), dtype=float32)
Tensor("discriminator_1/dense/BiasAdd:0", shape=(?, 1024), dtype=float32)
Tensor("discriminator_2/dense/BiasAdd:0", shape=(?, 1024), dtype=float32)


In [32]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter("output/", sess.graph)
    